In [1]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 googleads

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# Define the scopes required by the Google Ads API
SCOPES = ['https://www.googleapis.com/auth/adwords']

# Load client secrets JSON file
CLIENT_SECRETS_FILE = 'client_secrets.json'

# Define the token file where the access and refresh tokens will be stored
TOKEN_FILE = 'token.json'

# Function to handle OAuth2 flow
def get_credentials():
    creds = None

    # Check if token already exists
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    # If no valid credentials, start the OAuth2 flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for future use
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())

    return creds

# Get credentials
credentials = get_credentials()

# Now you can use these credentials to make requests to the Google Ads API

In [4]:
!pip install google-ads

In [5]:
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException
import os

# Set the path to your google-ads.yaml file
os.environ['GOOGLE_ADS_CONFIGURATION_FILE_PATH'] = 'google-ads.yaml'

# Create a GoogleAdsClient instance
client = GoogleAdsClient.load_from_storage()

In [8]:
# dismisses ALL recommendations that can be dismissed through the API (almost all)

# Updated CIDs as requested
customer_ids = [
    '0000000000', '0000000000', '0000000000', '0000000000', '0000000000'
]

# Query to fetch recommendations
query = """
    SELECT
        recommendation.resource_name,
        recommendation.type
    FROM recommendation
    WHERE recommendation.dismissed = false
"""

# Initialize services
ga_service = client.get_service("GoogleAdsService")
recommendation_service = client.get_service("RecommendationService")

# Loop through each customer ID
for customer_id in customer_ids:
    print(f"\nProcessing Customer ID: {customer_id}")
    
    try:
        # Fetch recommendations
        response = ga_service.search_stream(customer_id=customer_id, query=query)
        
        dismissed_count = 0
        has_recommendations = False
        
        # Process and dismiss recommendations
        for batch in response:
            has_recommendations = True
            for row in batch.results:
                try:
                    # Create dismiss operation
                    dismiss_operation = client.get_type("DismissRecommendationRequest")
                    dismiss_operation.customer_id = customer_id
                    dismiss_operation.operations = [{
                        "resource_name": row.recommendation.resource_name
                    }]
                    
                    # Execute dismiss
                    result = recommendation_service.dismiss_recommendation(
                        request=dismiss_operation
                    )
                    dismissed_count += 1
                except Exception as e:
                    print(f"Error dismissing recommendation: {e}")
        
        if not has_recommendations:
            print("No recommendations found to dismiss")
        else:
            print(f"Total recommendations dismissed: {dismissed_count}")
            
    except Exception as e:
        print(f"Error processing account: {e}")


Processing Customer ID: 4333537898
No recommendations found to dismiss

Processing Customer ID: 3644944593
No recommendations found to dismiss

Processing Customer ID: 2377507223
Total recommendations dismissed: 12

Processing Customer ID: 3708791639
Total recommendations dismissed: 14

Processing Customer ID: 5970229981
Total recommendations dismissed: 44

Processing Customer ID: 5505243116
Total recommendations dismissed: 23

Processing Customer ID: 6289037167
Total recommendations dismissed: 6

Processing Customer ID: 5796672495
Total recommendations dismissed: 2

Processing Customer ID: 5837408350
Total recommendations dismissed: 5

Processing Customer ID: 1227929101
Total recommendations dismissed: 11

Processing Customer ID: 7450438025
Total recommendations dismissed: 8

Processing Customer ID: 3344421286
Total recommendations dismissed: 40

Processing Customer ID: 5439423617
Total recommendations dismissed: 8

Processing Customer ID: 9001760045
Total recommendations dismissed: 

Request made: ClientCustomerId: 7435001146, Host: googleads.googleapis.com, Method: /google.ads.googleads.v18.services.RecommendationService/DismissRecommendation, RequestId: oSyo1w60i276iEmO9mIAEQ, IsFault: True, FaultMessage: Resource not found.


Error dismissing recommendation: (<_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Requested entity was not found."
	debug_error_string = "UNKNOWN:Error received from peer ipv4:172.217.1.10:443 {created_time:"2024-11-28T16:09:26.2296689+00:00", grpc_status:5, grpc_message:"Requested entity was not found."}"
>, <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Requested entity was not found."
	debug_error_string = "UNKNOWN:Error received from peer ipv4:172.217.1.10:443 {created_time:"2024-11-28T16:09:26.2296689+00:00", grpc_status:5, grpc_message:"Requested entity was not found."}"
>, errors {
  error_code {
    request_error: RESOURCE_NOT_FOUND
  }
  message: "Resource not found."
  location {
    field_path_elements {
      field_name: "operations"
      index: 0
    }
  }
  details {
  }
}
request_id: "oSyo1w60i276iEmO9mIAEQ"
, 'oSyo1w60i276iEmO9mIAEQ')
Total recommendations dismissed: 1

Processing Cu